# **Deep Learning Course**

## **Loss Functions and Multilayer Perceptrons (MLP)**

---

### **Student Information:**

- **Name:** *Zahra Maleki*
- **Student Number:** *400110009*

---

### **Assignment Overview**

In this notebook, we will explore various loss functions used in neural networks, with a specific focus on their role in training **Multilayer Perceptrons (MLPs)**. By the end of this notebook, you will have a deeper understanding of:
- Types of loss functions
- How loss functions affect the training process
- The relationship between loss functions and model optimization in MLPs

---

### **Table of Contents**

1. Introduction to Loss Functions
2. Types of Loss Functions
3. Multilayer Perceptrons (MLP)
4. Implementing Loss Functions in MLP
5. Conclusion

---



# 1.Introduction to Loss Functions 

In deep learning, **loss functions** play a crucial role in training models by quantifying the difference between the predicted outputs and the actual targets. Selecting the appropriate loss function is essential for the success of your model. In this assay, we will explore various loss functions available in PyTorch, understand their theoretical backgrounds, and provide you with a scaffolded class to experiment with these loss functions.

Before begining, let's train a simle MLP model using the **L1Loss** function. We'll return to this model later to experiment with different loss functions. We'll start by importing the necessary libraries and defining the model architecture.

First things first, let's talk about **L1Loss**.

### 1. L1Loss (`torch.nn.L1Loss`)
- **Description:** Also known as Mean Absolute Error (MAE), L1Loss computes the average absolute difference between the predicted values and the target values.
- **Use Case:** Suitable for regression tasks where robustness to outliers is desired.

Here is the mathematical formulation of L1Loss:
\begin{equation}
\text{L1Loss} = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{pred}_i} - y_{\text{true}_i}|
\end{equation}

Let's implement a simple MLP model using the L1Loss function.

In [111]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from tqdm import tqdm 
from torch.nn import NLLLoss, LogSoftmax
import torch.nn.functional as F
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from io import BytesIO
from PIL import Image
from sklearn.neural_network import MLPClassifier
# Don't be courious about Adam, it's just a fancy name for a fancy optimization algorithm

Here, we'll define a class called `SimpleMLP` that inherits from `nn.Module`. This class can have multiple layers, and we'll use the `nn.Sequential` module to define the layers of the model. The model will have the following architecture:

In [84]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU):
        super(SimpleMLP, self).__init__()
        # TODO: Define the layers of the MLP

        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))  
        layers.append(nn.ReLU()) 

        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim)) 
            layers.append(nn.ReLU()) 
    
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        if last_layer_activation_fn is not None:
            layers.append(last_layer_activation_fn())

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        # TODO: Define the forward pass of the MLP
        return self.model(x)

Now, let's define a class called `SimpleMLP_Loss` that has the following architecture:

In [85]:
class SimpleMLPTrainer:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        #TODO: Implement the training loop
        #Note: You should also print the training loss at each epoch, use tqdm for progress bar
        #Note: You should return the training loss at each epoch

        training_losses = []

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            self.model.train() 
            for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                
                loss = self.criterion(outputs, targets)
                
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item() * inputs.size(0) 

            epoch_loss /= len(train_loader.dataset)
            training_losses.append(epoch_loss)
            if (epoch % 10 ==0):
                print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        return training_losses

    def evaluate(self, val_loader):
        #TODO: Implement the evaluation loop
        #Note: You should return the validation loss and accuracy
        self.model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad(): 
            for inputs, targets in val_loader:
              
                outputs = self.model(inputs)
         
                loss = self.criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

                predictions = (torch.sigmoid(outputs) >= 0.5).float() 
                correct_predictions += (predictions == targets).sum().item()
                total_predictions += targets.size(0)

        val_loss /= len(val_loader.dataset)
        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")
        
        return val_loss, accuracy


Next, lets test our model using the L1Loss function. You'll use <span style="color:red">*Titanic Dataset*</span> to train the model.


In [41]:
# Load dataset
train_url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
data = pd.read_csv(train_url)

# Preprocessing (simple example)
data = data[['Pclass', 'Sex', 'Age', 'Fare', 'Survived']].dropna()
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# TODO: Convert the data to PyTorch tensors and create a DataLoader
# TODO: Split the data into training and validation sets
# TODO: Define the model, criterion, and optimizer

X = data[['Pclass', 'Sex', 'Age', 'Fare']].values
y = data['Survived'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1) 

X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1  

model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
criterion = nn.L1Loss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

trainer = SimpleMLPTrainer(model, criterion, optimizer)

num_epochs = 30
training_losses = trainer.train(train_loader, num_epochs=num_epochs)

val_loss, val_accuracy = trainer.evaluate(val_loader)

print(f"Final Validation Loss: {val_loss:.4f}")
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")

Epoch 1/30: 100%|██████████| 18/18 [00:00<00:00, 764.71it/s]


Epoch [1/30], Training Loss: 1.3003


Epoch 11/30: 100%|██████████| 18/18 [00:00<00:00, 445.17it/s]


Epoch [11/30], Training Loss: 0.4123


Epoch 21/30: 100%|██████████| 18/18 [00:00<00:00, 750.01it/s]


Epoch [21/30], Training Loss: 0.4102


Epoch 30/30: 100%|██████████| 18/18 [00:00<00:00, 1126.54it/s]

Validation Loss: 0.3916, Accuracy: 39.16%
Final Validation Loss: 0.3916
Final Validation Accuracy: 39.16%


<div style="text-align: center;"> <span style="color:red; font-size: 26px; font-weight: bold;">Let's train!</span> </div>

In [42]:
from torch.nn import L1Loss

# TODO: Train the model

# TODO: Evaluate the model

num_epochs = 300
batch_size = 32
learning_rate = 0.001


input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1  

model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, last_layer_activation_fn=None)
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

trainer = SimpleMLPTrainer(model, criterion, optimizer)

training_losses = trainer.train(train_loader, num_epochs=num_epochs)

val_loss, val_accuracy = trainer.evaluate(val_loader)

print(f"Final Validation Loss: {val_loss:.4f}")
print(f"Final Validation Accuracy: {val_accuracy * 100:.2f}%")


Epoch 1/300: 100%|██████████| 18/18 [00:00<00:00, 662.76it/s]


Epoch [1/300], Training Loss: 1.2978


Epoch 11/300: 100%|██████████| 18/18 [00:00<00:00, 1089.15it/s]


Epoch [11/300], Training Loss: 0.5713


Epoch 21/300: 100%|██████████| 18/18 [00:00<00:00, 883.92it/s]


...

Epoch [281/300], Training Loss: 0.4351


Epoch 291/300: 100%|██████████| 18/18 [00:00<00:00, 561.68it/s]


Epoch [291/300], Training Loss: 0.4376


Epoch 300/300: 100%|██████████| 18/18 [00:00<00:00, 1116.07it/s]

Validation Loss: 0.5122, Accuracy: 73.43%
Final Validation Loss: 0.5122
Final Validation Accuracy: 73.43%


---
# 2. Types of Loss Functions

PyTorch offers a variety of built-in loss functions tailored for different types of problems, such as regression, classification, and more. Below, we discuss several commonly used loss functions, their theoretical foundations, and typical use cases.

### 2. MSELoss (`torch.nn.MSELoss`)
- **Description:** Mean Squared Error (MSE) calculates the average of the squares of the differences between predicted and target values.
- **Use Case:** Commonly used in regression problems where larger errors are significantly penalized.

Here is boring math stuff for MSE:
\begin{equation}
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}
\end{equation}

<span style="color:red; font-size: 18px; font-weight: bold;">Warning:</span> Don't forget to reinitialize the model before experimenting with different loss functions.

In [43]:
from torch.nn import MSELoss

# TODO: Train the model

# TODO: Evaluate the model

model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, last_layer_activation_fn=None)
criterion = MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

trainer = SimpleMLPTrainer(model, criterion, optimizer)

training_losses = trainer.train(train_loader, num_epochs=700)

val_loss, val_accuracy = trainer.evaluate(val_loader)

print(f"Final Validation Loss with MSE: {val_loss:.4f}")
print(f"Final Validation Accuracy with MSE: {val_accuracy * 100:.2f}%")

Epoch 1/700: 100%|██████████| 18/18 [00:00<00:00, 704.29it/s]


Epoch [1/700], Training Loss: 3.7033


Epoch 11/700: 100%|██████████| 18/18 [00:00<00:00, 1124.96it/s]


Epoch [11/700], Training Loss: 0.2424


Epoch 21/700: 100%|██████████| 18/18 [00:00<00:00, 750.82it/s]


...

Epoch [681/700], Training Loss: 0.1338


Epoch 691/700: 100%|██████████| 18/18 [00:00<00:00, 748.09it/s]


Epoch [691/700], Training Loss: 0.1422


Epoch 700/700: 100%|██████████| 18/18 [00:00<00:00, 731.81it/s]

Validation Loss: 0.1741, Accuracy: 46.85%
Final Validation Loss with MSE: 0.1741
Final Validation Accuracy with MSE: 46.85%


### 3. NLLLoss (`torch.nn.NLLLoss`)
- **Description:** Negative Log-Likelihood Loss measures the likelihood of the target class under the predicted probability distribution.
- **Use Case:** Typically used in multi-class classification tasks, especially when combined with `log_softmax` activation.

Here is the mathematical formulation of NLLLoss:
\begin{equation}
\text{NLLLoss} = -\frac{1}{n} \sum_{i=1}^{n} \log(y_{i})
\end{equation}

I hope you note the logarithm in the formula. It's important! 

Why?

In this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [82]:
# Apply sigmoid activation for BCELoss
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.ReLU):
        super(SimpleMLP, self).__init__()
        
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())

        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        if last_layer_activation_fn is not None:
            layers.append(last_layer_activation_fn())
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)



class SimpleMLPTrainer:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        training_losses = []

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            self.model.train()
            for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                
                # Apply sigmoid activation for BCELoss
                probs = torch.sigmoid(outputs)
                
                # Calculate BCELoss
                loss = self.criterion(probs, targets)
                
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item() * inputs.size(0)

            epoch_loss /= len(train_loader.dataset)
            training_losses.append(epoch_loss)
            if (epoch % 10 == 0):
                print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        return training_losses

    def evaluate(self, val_loader):
        self.model.eval()
        val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = self.model(inputs)
                
                # Apply sigmoid activation for BCELoss
                probs = torch.sigmoid(outputs)
                
                loss = self.criterion(probs, targets)
                val_loss += loss.item() * inputs.size(0)

                predictions = (probs >= 0.5).float()
                correct_predictions += (predictions == targets).sum().item()
                total_predictions += targets.size(0)

        val_loss /= len(val_loader.dataset)
        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

        print(f"Final Validation Loss with BCELoss: {val_loss:.4f}")
        print(f"Final Validation Accuracy with BCELoss: {accuracy * 100:.2f}%")
        return val_loss, accuracy

In [83]:
# Run with relu activation function
from torch.nn import ReLU, NLLLoss, CrossEntropyLoss
# TODO: Train the model
# TODO: Evaluate the model

criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.01)

input_dim = X_train.shape[1]
hidden_dim = 16
output_dim = 1  
model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
trainer = SimpleMLPTrainer(model, criterion, optimizer)

training_losses = trainer.train(train_loader, num_epochs=100)
val_loss, val_accuracy = trainer.evaluate(val_loader)



Epoch 1/100: 100%|██████████| 18/18 [00:00<00:00, 1128.73it/s]


Epoch [1/100], Training Loss: 0.9708


Epoch 11/100: 100%|██████████| 18/18 [00:00<00:00, 1070.39it/s]


Epoch [11/100], Training Loss: 0.9708


Epoch 21/100: 100%|██████████| 18/18 [00:00<00:00, 1142.02it/s]


...

Epoch [81/100], Training Loss: 0.9708


Epoch 91/100: 100%|██████████| 18/18 [00:00<00:00, 745.07it/s]


Epoch [91/100], Training Loss: 0.9708


Epoch 100/100: 100%|██████████| 18/18 [00:00<00:00, 1128.44it/s]

Final Validation Loss with BCELoss: 0.8931
Final Validation Accuracy with BCELoss: 39.16%


In [ ]:
# Run with LogSoftmax activation function
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1, last_layer_activation_fn=nn.LogSoftmax):
        super(SimpleMLP, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        if last_layer_activation_fn is not None:
            layers.append(last_layer_activation_fn(dim=1))  
            
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)
    

class SimpleMLPTrainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer

    def manual_nll_loss(self, log_probs, targets):
        # Manually compute NLLLoss
        if targets.dim() > 1:
            targets = targets.squeeze()
        targets = targets.long()  # Convert to LongTensor if needed
        batch_size = log_probs.size(0)
        target_log_probs = log_probs[range(batch_size), targets]  # log prob for correct class
        return -target_log_probs.mean()

    def train(self, train_loader, num_epochs):
        training_losses = []
        for epoch in range(num_epochs):
            epoch_loss = 0.0
            self.model.train()
            for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                self.optimizer.zero_grad()
                log_probs = self.model(inputs) 
                loss = self.manual_nll_loss(log_probs, targets)  
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)
            epoch_loss /= len(train_loader.dataset)
            training_losses.append(epoch_loss)
            if (epoch % 10 == 0):
                print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")
        return training_losses

    def evaluate(self, val_loader):
  
        self.model.eval()
        val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
            
                targets = targets.squeeze().long()
                
                log_probs = self.model(inputs)
                loss = self.manual_nll_loss(log_probs, targets)
                val_loss += loss.item() * inputs.size(0)
                
                _, predicted = torch.max(log_probs, 1)
                
                correct_predictions += (predicted == targets).sum().item()
                total_predictions += targets.size(0)

        val_loss /= len(val_loader.dataset)
        
        accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")
        return val_loss, accuracy





In [63]:
# Run with LogSoftmax activation function
from torch.nn import NLLLoss

# TODO: Train the model

# TODO: Evaluate the model
model = SimpleMLP(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=2)  # 2 output nodes for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and evaluation
trainer = SimpleMLPTrainer(model, optimizer)

training_losses = trainer.train(train_loader, num_epochs=30)
val_loss, val_accuracy = trainer.evaluate(val_loader)

print(f"Final Validation Loss with manual NLLLoss: {val_loss:.4f}")
print(f"Final Validation Accuracy with manual NLLLoss: {val_accuracy :.2f}%")

Epoch 1/30: 100%|██████████| 18/18 [00:00<00:00, 562.34it/s]


Epoch [1/30], Training Loss: 1.6479


Epoch 11/30: 100%|██████████| 18/18 [00:00<00:00, 1115.06it/s]


Epoch [11/30], Training Loss: 0.6107


Epoch 21/30: 100%|██████████| 18/18 [00:00<00:00, 746.80it/s]


Epoch [21/30], Training Loss: 0.5735


Epoch 30/30: 100%|██████████| 18/18 [00:00<00:00, 552.18it/s]


Validation Loss: 0.6203, Accuracy: 61.54%
Final Validation Loss with manual NLLLoss: 0.6203
Final Validation Accuracy with manual NLLLoss: 61.54%


Your reason for your choice:

<div>
**Your answer here**
</div>

When using ReLU as the last layer activation function, the model output values are non-negative but unbounded, meaning they don't naturally fall within a probability distribution range (i.e., [0,1]). This output isn't optimal for binary or multi-class classification tasks where we want outputs representing class probabilities.

Impact of Using ReLU in the Last Layer: ReLU allows positive outputs but may produce values much greater than 1. When combined with loss functions like NLLLoss, this can lead to poor performance, as NLLLoss expects log-probabilities (typically from a log_softmax activation) rather than raw outputs.


For binary or multi-class classification, a more suitable solution is to use sigmoid (for binary) or softmax (for multi-class) in the last layer. These functions output values between 0 and 1, representing probabilities.



### 4. CrossEntropyLoss (`torch.nn.CrossEntropyLoss`)
- **Description:** Combines `LogSoftmax` and `NLLLoss` in one single class. It computes the cross-entropy loss between the target and the output logits.
- **Use Case:** Widely used for multi-class classification problems.

The mathematical formulation of CrossEntropyLoss is as follows:
\begin{equation}
  \text{CrossEntropy}(y, \hat{y}) = - \sum_{i=1}^{C} y_i \log\left(\frac{e^{\hat{y}_i}}{\sum_{j=1}^{C} e^{\hat{y}_j}}\right)
\end{equation}
  where:
  - \( C \) is the number of classes,
  - \( y_i \) is a one-hot encoded target vector (or a scalar class label),
  - \( \hat{y}_i \) represents the logits (unnormalized model outputs) for each class.
  
  In practice, `torch.nn.CrossEntropyLoss` expects raw logits as input and internally applies the softmax function to convert the logits into probabilities, followed by the negative log-likelihood computation.

- **Background:** Cross-entropy measures the difference between the true distribution \( y \) and the predicted distribution \( \hat{y} \). The function minimizes the negative log-probability assigned to the correct class, effectively penalizing predictions that deviate from the true class, making it a standard choice for classification tasks in deep learning.

Now, let's implement a class called `SimpleMLP_Loss` that has the following architecture:


In [ ]:
class SimpleMLP_Loss(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1):
        super(SimpleMLP_Loss, self).__init__()
        
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())

        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
class SimpleMLPTrainer_Loss:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        training_losses = []

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            self.model.train()
            for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                self.optimizer.zero_grad()
                outputs = self.model(inputs)  
                
                targets = targets.squeeze().long()
                
                loss = self.criterion(outputs, targets)
                
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item() * inputs.size(0)

            epoch_loss /= len(train_loader.dataset)
            training_losses.append(epoch_loss)
            if (epoch % 10 == 0):
                print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        return training_losses

    def evaluate(self, val_loader):
        self.model.eval()
        val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = self.model(inputs)  
                
                targets = targets.squeeze().long()
                
                loss = self.criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                
                _, predictions = torch.max(outputs, 1)
                correct_predictions += (predictions == targets).sum().item()
                total_predictions += targets.size(0)

        val_loss /= len(val_loader.dataset)
        accuracy = correct_predictions / total_predictions * 100 if total_predictions > 0 else 0

        print(f"Final Validation Loss with CrossEntropyLoss: {val_loss:.4f}")
        print(f"Final Validation Accuracy with CrossEntropyLoss: {accuracy:.2f}%")
        return val_loss, accuracy


In [92]:
from torch.nn import CrossEntropyLoss

#TODO Train the model

#TODO Evaluate the model
input_dim = X_train.shape[1]
hidden_dim =16
output_dim = 2   

model = SimpleMLP_Loss(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)
trainer = SimpleMLPTrainer_Loss(model, criterion, optimizer)

training_losses = trainer.train(train_loader, num_epochs=100)
val_loss, val_accuracy = trainer.evaluate(val_loader)


Epoch 1/100: 100%|██████████| 18/18 [00:00<00:00, 455.52it/s]


Epoch [1/100], Training Loss: 2.1591


Epoch 11/100: 100%|██████████| 18/18 [00:00<00:00, 747.76it/s]


Epoch [11/100], Training Loss: 0.5732


Epoch 21/100: 100%|██████████| 18/18 [00:00<00:00, 439.08it/s]


...

Epoch [81/100], Training Loss: 0.4748


Epoch 91/100: 100%|██████████| 18/18 [00:00<00:00, 448.81it/s]


Epoch [91/100], Training Loss: 0.4798


Epoch 100/100: 100%|██████████| 18/18 [00:00<00:00, 551.23it/s]


Final Validation Loss with CrossEntropyLoss: 0.5468
Final Validation Accuracy with CrossEntropyLoss: 73.43%



### 5. KLDivLoss (`torch.nn.KLDivLoss`)
- **Description:** Kullback-Leibler Divergence Loss measures how one probability distribution diverges from a second, reference distribution. Unlike other loss functions that focus on classification, KL divergence specifically compares the relative entropy between two distributions. It quantifies the information loss when using the predicted distribution to approximate the true distribution. 

- **Mathematical Function:**
\begin{equation}
  \text{KL}(P \parallel Q) = \sum_{i=1}^{C} P(i) \left( \log P(i) - \log Q(i) \right)
\end{equation}
  where:
  - \( P \) is the target (true) probability distribution,
  - \( Q \) is the predicted distribution (often the output of `log_softmax`),
  - \( C \) is the number of classes.

  KL divergence is always non-negative, and it equals zero if the two distributions are identical. The loss function expects the model's output to be in the form of log-probabilities (using `log_softmax`) and compares this against a target probability distribution, which is typically a normalized distribution (using softmax).

- **Use Case:** KLDivLoss is frequently used in:
  - **Variational Autoencoders (VAEs):** In VAEs, KL divergence is used to measure how much the learned latent space distribution deviates from a prior distribution (often Gaussian).
  - **Knowledge Distillation:** In teacher-student models, KL divergence is used to transfer the "soft" knowledge from a teacher model to a student model by comparing their output probability distributions.
  - **Reinforcement Learning:** It can be used to update policies while minimizing the divergence from a previous policy.

- **Background:** Kullback-Leibler divergence, a core concept in information theory, measures the inefficiency of assuming the predicted distribution \( Q \) when the true distribution is \( P \). It is asymmetric, meaning that \( KL(P \parallel Q) \neq KL(Q \parallel P) \), so the direction of the comparison matters.

Again, in this part, run your training with Relu at last layer. <span style="color:red; font-weight: bold;">Discuss </span> and explain the difference between the results of the two models. Find a proper solution to the problem.


In [98]:
class SimpleMLP_KL(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers=1):
        super(SimpleMLP_KL, self).__init__()
        
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())

        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.LogSoftmax(dim=1))  
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class SimpleMLPTrainer_KL:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def train(self, train_loader, num_epochs):
        training_losses = []

        for epoch in range(num_epochs):
            epoch_loss = 0.0
            self.model.train()
            for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                self.optimizer.zero_grad()
                outputs = self.model(inputs)  
                target_probs = torch.softmax(targets, dim=1)
                
                loss = self.criterion(outputs, target_probs)
                
                loss.backward()
                self.optimizer.step()
                
                epoch_loss += loss.item() * inputs.size(0)

            epoch_loss /= len(train_loader.dataset)
            training_losses.append(epoch_loss)
            if (epoch % 10 == 0):
                print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

        return training_losses

    def evaluate(self, val_loader):
        self.model.eval()
        val_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = self.model(inputs)  # Log probabilities

                # Convert targets to one-hot encoding and then to a probability distribution
                targets = F.one_hot(targets.squeeze().long(), num_classes=outputs.size(1)).float()
                
                # Calculate KLDivLoss
                loss = self.criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)

                # Convert log-probabilities to predicted class
                predictions = torch.argmax(outputs.exp(), dim=1)  # Convert to probabilities and get max
                correct_predictions += (predictions == targets.argmax(dim=1)).sum().item()
                total_predictions += targets.size(0)

        val_loss /= len(val_loader.dataset)
        accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

        print(f"Final Validation Loss with KLDivLoss: {val_loss:.4f}")
        print(f"Final Validation Accuracy with KLDivLoss: {accuracy:.2f}%")
        return val_loss, accuracy


In [ ]:
# Run with LogSoftmax activation function
from torch.nn import NLLLoss

# TODO: Train the model

# TODO: Evaluate the model

model = SimpleMLP_KL(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
criterion = nn.KLDivLoss(reduction='batchmean')  
optimizer = optim.Adam(model.parameters(), lr=0.001)

trainer = SimpleMLPTrainer_KL(model, criterion, optimizer)
training_losses = trainer.train(train_loader, num_epochs=80)
val_loss, val_accuracy = trainer.evaluate(val_loader)

Epoch 1/80: 100%|██████████| 18/18 [00:00<00:00, 478.64it/s]


Epoch [1/80], Training Loss: 4.0629


Epoch 11/80: 100%|██████████| 18/18 [00:00<00:00, 1123.59it/s]


Epoch [11/80], Training Loss: 1.4018


Epoch 21/80: 100%|██████████| 18/18 [00:00<00:00, 1096.95it/s]


...

Epoch [61/80], Training Loss: 1.3874


Epoch 71/80: 100%|██████████| 18/18 [00:00<00:00, 1124.09it/s]


Epoch [71/80], Training Loss: 1.3868


Epoch 80/80: 100%|██████████| 18/18 [00:00<00:00, 1122.34it/s]

Final Validation Loss with KLDivLoss: 0.6939
Final Validation Accuracy with KLDivLoss: 53.15%


Your reason for your choice:

<div>
**Your answer here**
</div>

To use KLDivLoss effectively, we need to ensure that the model outputs log-probabilities (using log_softmax in the final layer) and that the targets are formatted as probability distributions (using softmax). Using KLDivLoss with ReLU in the final layer will not work as expected, because ReLU does not produce a normalized probability distribution.

With ReLU Activation: If using ReLU, the model cannot produce normalized probabilities, so KLDivLoss becomes meaningless because it compares distributions.

With LogSoftmax Activation: Using log_softmax produces log-probabilities, which allows KLDivLoss to calculate divergence between the predicted and target distributions effectively.

### 6. CosineEmbeddingLoss (`torch.nn.CosineEmbeddingLoss`)
- **Description:** Measures the cosine similarity between two input tensors, `x1` and `x2`, and computes the loss based on a label `y` that indicates whether the tensors should be similar (`y = 1`) or dissimilar (`y = -1`). Cosine similarity focuses on the angle between vectors, disregarding their magnitude.

- **Mathematical Function:** 
\begin{equation}
  \text{CosineEmbeddingLoss}(x1, x2, y) = 
  \begin{cases} 
  1 - \cos(x_1, x_2), & \text{if } y = 1 \\
  \max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
  \end{cases}
\end{equation}
  where $ \cos(x_1, x_2) $ is the cosine similarity between the two vectors, and `margin` is a threshold that determines how dissimilar the vectors should be.

- **Use Case:** Commonly used in tasks like face verification, image similarity, and other scenarios where the relative orientation of vectors (angle) is more important than their length, such as in embeddings and metric learning.

- **Background:** Cosine similarity compares the directional alignment of vectors, making it ideal for high-dimensional data where the magnitude may not be as informative. This loss is particularly useful when training models to learn meaningful embeddings that capture semantic similarity.

You'll become more fimiliar with this loss function in future.

---

# Regularization in Machine Learning

## Introduction

Regularization is a fundamental technique in machine learning that helps prevent overfitting by adding a penalty to the loss function. This penalty discourages the model from becoming too complex, ensuring better generalization to unseen data. In this notebook, you will explore the concepts of regularization, understand different types of regularization techniques, and apply them using Python's popular libraries.

## What is Regularization?

Regularization involves adding a regularization term to the loss function used to train machine learning models. This term imposes a constraint on the model's coefficients, effectively reducing their magnitude. By doing so, regularization helps in:

- **Preventing Overfitting:** Ensures the model does not become too tailored to the training data.
- **Improving Generalization:** Enhances the model's performance on new, unseen data.
- **Feature Selection:** Especially in L1 regularization, it can drive some coefficients to zero, effectively selecting important features.

## Types of Regularization

There are several types of regularization techniques, each imposing different constraints on the model's parameters:

### 1. L1 Regularization (Lasso)

L1 regularization adds the absolute value of the magnitude of coefficients as a penalty term to the loss function. It can lead to sparse models where some feature coefficients are exactly zero.

### 2. L2 Regularization (Ridge)

L2 regularization adds the squared magnitude of coefficients as a penalty term to the loss function. It tends to shrink the coefficients evenly but does not set them to zero.

### 3. Elastic Net

Elastic Net combines both L1 and L2 regularization penalties. It balances the benefits of both Lasso and Ridge methods, allowing for feature selection and coefficient shrinkage.

## Homework Time!
Import Iris dataset from sklearn.datasets and apply ridge regression with different alpha values. Then, create a gif that shows the changes of the classification boundary with respect to alpha values.

Import the libs that you need and start coding!

In [104]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from PIL import Image
from io import BytesIO
import imageio
import warnings


# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

Load the Iris dataset and select Setosa and Versicolor classes

In [106]:
# 1. Load and Prepare the Iris Dataset

# Select only two classes for binary classification (Setosa and Versicolor)

# Select two features for 2D visualization (Sepal Length and Petal Length)

# Split into training and testing sets

iris = load_iris()
X = iris.data
y = iris.target

selected_classes = (y == 0) | (y == 1)
X = X[selected_classes]
y = y[selected_classes]

X = X[:, [0, 2]]  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)
print("Sample of standardized training features:\n", X_train[:5])
print("Sample of training labels:\n", y_train[:5])


Training set shape: (70, 2)
Testing set shape: (30, 2)
Sample of standardized training features:
 [[1.94114606 1.29689348]
 [1.62587872 1.22635537]
 [2.2564134  1.43796969]
 [1.94114606 1.08527916]
 [2.09877973 1.36743159]]
Sample of training labels:
 [1 1 1 1 1]


Define Function to Plot Decision Boundary

In [113]:
def plot_decision_boundary(model, X, y, alpha):
    # Define the grid over the feature space
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    # Predict over the grid
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid)
    Z = Z.reshape(xx.shape)

    # Create a figure
    fig, ax = plt.subplots(figsize=(6, 5))

    # Plot the decision boundary
    ax.contourf(xx, yy, Z, alpha=0.3, levels=[-0.1, 0.1, 1.1], colors=['blue', 'red'])

    # Scatter plot of the training data
    scatter = ax.scatter(
        X[:, 0], X[:, 1], c=y, cmap='bwr', edgecolor='k', s=50
    )

    # Title and labels
    ax.set_title(f'MLP Decision Boundary (alpha={alpha})')
    ax.set_xlabel('Sepal Length (standardized)')
    ax.set_ylabel('Petal Length (standardized)')

    # Remove axes for clarity
    ax.set_xticks([])
    ax.set_yticks([])

    # Tight layout
    plt.tight_layout()

    # Save the plot to a BytesIO object
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

Train MLP with Varying Alpha Values and Collect Images

In [116]:
def create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons):
    # List to store images
    images = []

    for idx, alpha in enumerate(alpha_values):
        print(f"Processing alpha={alpha:.4f} ({idx + 1}/{len(alpha_values)})")

        # Create and train the MLP
        mlp = MLPClassifier(hidden_layer_sizes=(n_neurons,), alpha=alpha, max_iter=1000, random_state=42)
        mlp.fit(X_train, y_train)

        # Plot decision boundary and get the image
        img = plot_decision_boundary(mlp, X_train, y_train, alpha)
        images.append(img)

    # Save the images as a GIF
    gif_filename = 'mlp_classification_boundaries_Rosa.gif'
    images[0].save(
        gif_filename,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f"GIF saved as '{gif_filename}'")

    return gif_filename


## RUN

In [117]:

# Use np.logspace to generate alpha values from 0.01 to 400
alpha_values = np.logspace(np.log10(0.01), np.log10(400), num=30)
# Define the number of neurons in the hidden layer
n_neurons = 10  # Example number of neurons, adjust as desired

# Create the decision boundary GIF
gif_filename = create_decision_boundary_gif(alpha_values, X_train, y_train, n_neurons)

Processing alpha=0.0100 (1/30)
Processing alpha=0.0144 (2/30)
Processing alpha=0.0208 (3/30)
Processing alpha=0.0299 (4/30)
Processing alpha=0.0431 (5/30)
Processing alpha=0.0622 (6/30)
Processing alpha=0.0896 (7/30)
Processing alpha=0.1291 (8/30)
Processing alpha=0.1860 (9/30)
Processing alpha=0.2681 (10/30)
Processing alpha=0.3863 (11/30)
Processing alpha=0.5567 (12/30)
Processing alpha=0.8022 (13/30)
Processing alpha=1.1561 (14/30)
Processing alpha=1.6660 (15/30)
Processing alpha=2.4009 (16/30)
Processing alpha=3.4599 (17/30)
Processing alpha=4.9861 (18/30)
Processing alpha=7.1854 (19/30)
Processing alpha=10.3548 (20/30)
Processing alpha=14.9223 (21/30)
Processing alpha=21.5043 (22/30)
Processing alpha=30.9897 (23/30)
Processing alpha=44.6591 (24/30)
Processing alpha=64.3578 (25/30)
Processing alpha=92.7456 (26/30)
Processing alpha=133.6549 (27/30)
Processing alpha=192.6090 (28/30)
Processing alpha=277.5673 (29/30)
Processing alpha=400.0000 (30/30)
GIF saved as 'mlp_classification_b

Your gif should look like this:

<div style="text-align: center;">

### **Multilayer Perceptron Classification Boundaries**

![Classification Boundaries](mlp_classification_boundaries_example.gif)

*Figure 1: Demonstration of classification boundaries created by a Multilayer Perceptron (MLP) model.*

</div>

